In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

import pickle

from sklearn.svm import SVC

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk import pos_tag

# ! pip install stopwords
# ! pip install stop_words
# ! pip install gensim
# ! pip install python-Levenshtein
# ! pip install pickle-mixin

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')
# nltk.download('wordnet')

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
MAX_BOW_SIZE = 1000

In [5]:
def analysis(labels, predictions):
    print("Report Classification: \n", classification_report(labels, predictions, target_names=['positive', 'negative']))
    print("Matrix Confusion: \n", confusion_matrix(labels, predictions))
    print("Accuracy: \n", accuracy_score(labels,predictions))

# Part1

In [6]:
data_set = pd.read_csv('dataset.csv')

X = data_set['comment']
Y = data_set['sentiment']


# label binarization
label_binarizer = LabelBinarizer()
Y = label_binarizer.fit_transform(Y)
Y = np.ravel(Y)

# Part 2, 3

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=5)

In [8]:
stop_words = list(get_stop_words('en'))
nltk_words = list(stopwords.words('english'))
stop_words.extend(nltk_words)

def lemmatize(text):
    list_pos = 0
    cleaned_str = ''
    lmtzr = WordNetLemmatizer()
    
    tagged_words = pos_tag(text)
    for word in tagged_words:
        if 'v' in word[1].lower():
            lemma = lmtzr.lemmatize(word[0], pos='v')
        else:
            lemma = lmtzr.lemmatize(word[0], pos='n')
        if list_pos == 0:
            cleaned_str = lemma
        else:
            cleaned_str = cleaned_str + ' ' + lemma
        list_pos += 1
    return cleaned_str

def clean(text):
    text = str(text).lower() #lowercase
    text = re.sub(r'\bid\b', 'i would', text) #start abbreviation
    text = re.sub(r'\bive\b', 'i have', text)
    text = re.sub(r'\bim\b', 'i am', text)
    text = re.sub(r'\bcant\b', 'can not', text)
    text = re.sub(r'\bdont\b', 'do not', text)
    text = re.sub(r'\bwont\b', 'will not', text)
    text = re.sub(r'\bthats\b', 'that is', text) #end abbreviation
    text = re.sub('[0-9]+', '', text) # delete numbers
    text = re.sub(r'[^\x00-\x7F]+',' ', text) #remove non-ascii
    text = re.sub('[<>{}=~.,،:\\!?\\-()\\[\\]#/@"]+|[_x000D_]+|\u200c+|[\r\n]', ' ', text) #remove punctuations
    word_list = nltk.word_tokenize(text)
    text = lemmatize(word_list)
    word_list = text.split()
    word_list = list(filter(lambda word: word not in stop_words, word_list)) # delete stopwords
    word_list = [w for w in word_list if len(w)>1] # delete len = 1
    return word_list


### BOW

In [9]:
count_vectorizer = CountVectorizer(tokenizer=lambda text: clean(text), max_features=MAX_BOW_SIZE)

In [10]:
cv_X_train = count_vectorizer.fit_transform(X_train) #fit only over train data 
cv_X_test = count_vectorizer.transform(X_test) #apply not fit!

### W2V

In [11]:
corpus_text_train = '\n'.join(X_train) # just for X_train fits
sentences_train = corpus_text_train.split('\n')
sentences_train = [clean(line) for line in sentences_train]

model = Word2Vec(sentences_train, window=5, min_count=3, workers=4)
vectors = model.wv

w2v_sentences_train = []
for index in range(len(sentences_train)):
    temp = []
    for word in sentences_train[index]:
        try:
            temp.append(vectors[word])
        except:
            pass
    w2v_sentences_train.append(np.mean(temp, axis=0))
    
    
corpus_text_test = '\n'.join(X_test) # just apply for X_test
sentences_test = corpus_text_test.split('\n')
sentences_test = [clean(line) for line in sentences_test]

w2v_sentences_test = []
for index in range(len(sentences_test)):
    temp = []
    for word in sentences_test[index]:
        try:
            temp.append(vectors[word])
        except:
            pass
    w2v_sentences_test.append(np.mean(temp, axis=0))

### knn: BOW

In [12]:
k_range = list(range(1,31))
weight_options = ["uniform", "distance"]

param_grid = dict(n_neighbors = k_range, weights = weight_options)
knn = KNeighborsClassifier()

clf = GridSearchCV(knn, param_grid, scoring ='accuracy')
clf.fit(cv_X_train,Y_train)

print("-------KNN-------:")
print("Best parameters set found on development set:")
print (clf.best_params_)
print (clf.best_estimator_)
Y_test_pred = clf.predict(cv_X_test)
analysis(Y_test, Y_test_pred)

-------KNN-------:
Best parameters set found on development set:
{'n_neighbors': 28, 'weights': 'distance'}
KNeighborsClassifier(n_neighbors=28, weights='distance')
Report Classification: 
               precision    recall  f1-score   support

    positive       0.66      0.69      0.68      4467
    negative       0.68      0.66      0.67      4533

    accuracy                           0.67      9000
   macro avg       0.67      0.67      0.67      9000
weighted avg       0.67      0.67      0.67      9000

Matrix Confusion: 
 [[3075 1392]
 [1559 2974]]
Accuracy: 
 0.6721111111111111


In [13]:
filename = 'Bow_Knn.pkl'
pickle.dump(clf, open(filename, 'wb'))

### knn: W2V

In [14]:
k_range = list(range(1,31))
weight_options = ["uniform", "distance"]

param_grid = dict(n_neighbors = k_range, weights = weight_options)
knn = KNeighborsClassifier()

clf = GridSearchCV(knn, param_grid, scoring ='accuracy')
clf.fit(w2v_sentences_train,Y_train)

print("-------KNN-------:")
print("Best parameters set found on development set:")
print (clf.best_params_)
print (clf.best_estimator_)
Y_test_pred = clf.predict(w2v_sentences_test)
analysis(Y_test, Y_test_pred)

-------KNN-------:
Best parameters set found on development set:
{'n_neighbors': 24, 'weights': 'distance'}
KNeighborsClassifier(n_neighbors=24, weights='distance')
Report Classification: 
               precision    recall  f1-score   support

    positive       0.80      0.84      0.82      4467
    negative       0.83      0.79      0.81      4533

    accuracy                           0.82      9000
   macro avg       0.82      0.82      0.82      9000
weighted avg       0.82      0.82      0.82      9000

Matrix Confusion: 
 [[3748  719]
 [ 933 3600]]
Accuracy: 
 0.8164444444444444


In [15]:
filename = 'W2v_Knn.pkl'
pickle.dump(clf, open(filename, 'wb'))

### logistic regression: Bow


In [16]:
grid_values = {'penalty': ['l2'], 'C': [0.1, 1,10, 100]}
clf = GridSearchCV(LogisticRegression(), param_grid=grid_values, scoring = 'accuracy')

clf.fit(cv_X_train,Y_train)

print("-------LR-------:")
print("Best parameters set found on development set:")
print (clf.best_params_)
print (clf.best_estimator_)
Y_test_pred = clf.predict(cv_X_test)
analysis(Y_test, Y_test_pred)

-------LR-------:
Best parameters set found on development set:
{'C': 0.1, 'penalty': 'l2'}
LogisticRegression(C=0.1)
Report Classification: 
               precision    recall  f1-score   support

    positive       0.87      0.85      0.86      4467
    negative       0.86      0.88      0.87      4533

    accuracy                           0.86      9000
   macro avg       0.86      0.86      0.86      9000
weighted avg       0.86      0.86      0.86      9000

Matrix Confusion: 
 [[3806  661]
 [ 560 3973]]
Accuracy: 
 0.8643333333333333


In [17]:
filename = 'Bow_LR.pkl'
pickle.dump(clf, open(filename, 'wb'))

### logistic regression: W2V


In [18]:
grid_values = {'penalty': ['l2'], 'C': [0.1, 1,10, 100]}
clf = GridSearchCV(LogisticRegression(), param_grid=grid_values, scoring = 'accuracy')

clf.fit(w2v_sentences_train,Y_train)

print("-------LR-------:")
print("Best parameters set found on development set:")
print (clf.best_params_)
print (clf.best_estimator_)
Y_test_pred = clf.predict(w2v_sentences_test)
analysis(Y_test, Y_test_pred)

-------LR-------:
Best parameters set found on development set:
{'C': 1, 'penalty': 'l2'}
LogisticRegression(C=1)
Report Classification: 
               precision    recall  f1-score   support

    positive       0.86      0.85      0.85      4467
    negative       0.85      0.86      0.86      4533

    accuracy                           0.86      9000
   macro avg       0.86      0.86      0.86      9000
weighted avg       0.86      0.86      0.86      9000

Matrix Confusion: 
 [[3787  680]
 [ 620 3913]]
Accuracy: 
 0.8555555555555555


In [19]:
filename = 'W2V_LR.pkl'
pickle.dump(clf, open(filename, 'wb'))

### svm: BOW

In [20]:
tuned_parameters = [{'kernel': ['poly', 'rbf'], 'C': [1, 10]}]

clf = GridSearchCV(SVC(), tuned_parameters, scoring ='accuracy')
clf.fit(cv_X_train, Y_train)

print("-------SVM-------:")
print("Best parameters set found on development set:")
print(clf.best_params_)
Y_test_pred = clf.predict(cv_X_test)
analysis(Y_test, Y_test_pred)


-------SVM-------:
Best parameters set found on development set:
{'C': 1, 'kernel': 'rbf'}
Report Classification: 
               precision    recall  f1-score   support

    positive       0.87      0.84      0.86      4467
    negative       0.85      0.88      0.86      4533

    accuracy                           0.86      9000
   macro avg       0.86      0.86      0.86      9000
weighted avg       0.86      0.86      0.86      9000

Matrix Confusion: 
 [[3772  695]
 [ 557 3976]]
Accuracy: 
 0.8608888888888889


In [21]:
filename = 'Bow_svm.pkl'
pickle.dump(clf, open(filename, 'wb'))

### svm: W2V

In [22]:
tuned_parameters = [{'kernel': ['poly','rbf'], 'C': [1, 10]}]

clf = GridSearchCV(SVC(), tuned_parameters, scoring ='accuracy')
clf.fit(w2v_sentences_train, Y_train)

print("-------SVM-------:")
print("Best parameters set found on development set:")
print(clf.best_params_)
Y_test_pred = clf.predict(w2v_sentences_test)
analysis(Y_test, Y_test_pred)


-------SVM-------:
Best parameters set found on development set:
{'C': 10, 'kernel': 'rbf'}
Report Classification: 
               precision    recall  f1-score   support

    positive       0.87      0.85      0.86      4467
    negative       0.86      0.87      0.86      4533

    accuracy                           0.86      9000
   macro avg       0.86      0.86      0.86      9000
weighted avg       0.86      0.86      0.86      9000

Matrix Confusion: 
 [[3818  649]
 [ 589 3944]]
Accuracy: 
 0.8624444444444445


In [23]:
filename = 'W2V_svm.pkl'
pickle.dump(clf, open(filename, 'wb'))